In [1]:
import os
import pymssql
import pandas as pd
import sys
sys.path.append('../../')
# cwd = os.getcwd()
# print(cwd) 
from my_lib.hmm import hmm_util as hmm_u
from my_lib.GOP_connection import GOPserver as gop
import matplotlib.pyplot as plt
import datetime
import numpy as np
# Progress bar
from tqdm import tqdm

In [78]:
gop_sv = gop.GOPserver()
n_parts = 100
data_path = "../../data/"

In [79]:
dt_now = datetime.datetime.now()
d = pd.date_range("2014-01-01",dt_now)
sp_time_range = np.array_split(d, n_parts)

In [80]:
def pull_data(sql_str, sp_time_range):
    sp_time_range = [x for x in sp_time_range if not x.empty]
    df_t = pd.DataFrame()
    for sp in tqdm(sp_time_range, desc="Trainnig process", ncols=100):
        sql = sql_str.format(sp[0], sp[-1])
        df = pd.read_sql(sql,gop_sv.conn)
        df["timestamp"] = df["Fecha"] + " " + df["Hora"]
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df = df.groupby('timestamp').sum()
        df.sort_index(inplace=True)
        df = hmm_u.pivot_DF_using_dates_and_hours(df)
        df_t = df_t.append(df)
    return df_t
# df["unique"] = df["Central"] + df["Unidad"] + df["timestamp"]
# df.drop_duplicates(subset=["unique"], inplace=True)

In [81]:
sql_str = "SELECT t.Central, t.Unidad, t.MV_Validado, " +\
              "t.Fecha, t.Hora FROM SIVO.dbo.DV_Generacion t "+ \
              "where t.MV_Validado > 0 and t.Fecha between '{0}' and '{1}'"

sql_file = 'Bornes_P_Total.pkl'
file_path = data_path + sql_file
    
if not os.path.exists(file_path):
    df_t = pull_data(sql_str, sp_time_range)
else:
    df_t = pd.read_pickle(file_path)
    d = pd.date_range(df_t.index[-1], dt_now)
    sp_new = np.array_split(d, n_parts + 1)
    df_n = pull_data(sql_str, sp_new)
    df_t = df_t.append(df_n)

print("[{0: <30s}] Pulling data from {1} to {2} ".format(sql_file, df_t.index[0], df_t.index[-1])) 
df_t.to_pickle(data_path + 'Bornes_P_Total.pkl')

Trainnig process: 100%|███████████████████████████████████████████| 101/101 [01:51<00:00,  1.10s/it]


[Bornes_P_Total.pkl            ] Pulling data from 2014-01-01 to 2018-05-22 


In [9]:
file_path = data_path + 'Bornes_P_Total.pkl'
not os.path.exists(file_path) 

False

In [37]:
sp_time_range

[DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-09', '2014-01-10', '2014-01-11', '2014-01-12'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-17', '2014-01-18', '2014-01-19', '2014-01-20'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-21', '2014-01-22', '2014-01-23', '2014-01-24'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-25', '2014-01-26', '2014-01-27', '2014-01-28'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-01-29', '2014-01-30', '2014-01-31', '2014-02-01'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-02-02', '2014-02-03', '2014-02-04', '2014-02-05'], dtype='datetime64[ns]', freq='D'),
 DatetimeIndex(['2014-02-06'